### Jewel Osco meat prices

In [4]:
from bs4 import BeautifulSoup
import requests

header = { 
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36 Edg/137.0.3296.93'
    }
page_url = 'https://www.jewelosco.com/aisle-vs/meat-seafood/meat-essentials.html?sort=price'

# check request
page_to_scrape = requests.get(page_url, headers=header)
if page_to_scrape.status_code != 200:
    print(f"Failed to fetch page. Status code: {page_to_scrape.status_code}")
    exit()

# scrape data from url
soup = BeautifulSoup(page_to_scrape.content, 'lxml')
products = soup.findAll('a', attrs={'data-qa':'prd-itm-pttl'})

if not products:
    print("No products found. Possible issues:")
    print("- The page structure may have changed")
    print("- The content might be loaded dynamically with JavaScript")
    print("- The website might be blocking your scraper")
else:
    for product in products:
        print(product.text.strip())